In [157]:
W = [
    [[0, 16, 16, 10],
     [0,  0,  8,  8],
     [0,  0,  0,  4],
     [0,  0,  0,  0]],
    
    [[0, 5, 4,  7, 6,  8],
     [0, 0, 7, 10, 9, 11],
     [0, 0, 0,  7, 6,  8],
     [0, 0, 0,  0, 5,  9],
     [0, 0, 0,  0, 0,  8],
     [0, 0, 0,  0, 0,  0]],
]

N = [
    'ABCD',
    'ABCDEF',
]

# WPGMA

In [158]:
class node:
    name = ''
    final = True
    full_dist = 0
    node_dist = 0
    child_nodes = []
    
    def __init__(self, name, final=True, child_node_1=None, child_node_2=None, full_dist=0):
        self.name = name
        self.final = final
        self.full_dist = full_dist
        
        if not final:
            child_node_1.node_dist = self.full_dist - child_node_1.full_dist
            child_node_2.node_dist = self.full_dist - child_node_2.full_dist
            self.child_nodes = [child_node_1, child_node_2]
    
    def __str__(self):
        if self.final:
            return self.name + ':' + str(round(self.node_dist, 2))
        else:
            str_child_1 = str(self.child_nodes[0])
            str_child_2 = str(self.child_nodes[1])
            if self.node_dist != 0:
                return '(' + str_child_1 + ',' + str_child_2 + '):' + str(round(self.node_dist, 2))
            else:
                return '(' + str_child_1 + ',' + str_child_2 + ')'

def find_min(WM):
    L = len(WM)
    
    cur_i = 0
    cur_j = 1
    cur_min = WM[cur_i][cur_j]

    for i in range(L):
        for j in range(i + 1, L):
            if WM[i][j] < cur_min:
                cur_i = i
                cur_j = j
                cur_min = WM[cur_i][cur_j]
    return cur_i, cur_j, cur_min 

def WPGMA(weight_matrix, names):
    res = [node(n) for n in names]

    while len(weight_matrix) > 1:
        i, j, r_min = find_min(weight_matrix)

        full_dist = r_min / 2
            
        names[i] = names[i] + names[j]
        names.pop(j)
        
        res[i] = node(names[i], False, res[i], res[j], full_dist)
        res.pop(j)
        
        L = len(weight_matrix)
                
        # update i row
        for k in range(i + 1, L):
            if k != j:
                
                temp_w = 0
                if k < j:
                    temp_w = weight_matrix[k][j]
                else:
                    temp_w = weight_matrix[j][k]
                                
                weight_matrix[i][k] = (weight_matrix[i][k] + temp_w) / 2
        
        # update i column
        for k in range(0, i):
            weight_matrix[k][i] = (weight_matrix[k][i] + weight_matrix[k][j]) / 2
        
        for k in range(L):
            weight_matrix[k].pop(j)
        
        weight_matrix.pop(j)
    
    print(res[0])

def start_test(test):
    names_test = [i for i in N[test]]
    weight_matrix_test = [w.copy() for w in W[test]]

    WPGMA(weight_matrix_test, names_test)

Test 1.

In [159]:
test = int(input()) - 1
start_test(test)

1
(A:7.25,(B:4.0,(C:2.0,D:2.0):2.0):3.25)


Test 2.

In [160]:
test = int(input()) - 1
start_test(test)

2
((((A:2.0,C:2.0):1.0,B:3.0):1.0,(D:2.5,E:2.5):1.5):0.5,F:4.5)


# UPGMA

In [161]:
class w_node:
    name = ''
    final = True
    full_dist = 0
    node_dist = 0
    weight = 1
    child_nodes = []
    
    def __init__(self, name, final=True, child_node_1=None, child_node_2=None, full_dist=0):
        self.name = name
        self.final = final
        self.full_dist = full_dist
        
        if not final:
            child_node_1.node_dist = self.full_dist - child_node_1.full_dist
            child_node_2.node_dist = self.full_dist - child_node_2.full_dist
            self.weight = child_node_1.weight + child_node_2.weight
            self.child_nodes = [child_node_1, child_node_2]
    
    def __str__(self):
        if self.final:
            return self.name + ':' + str(round(self.node_dist, 2))
        else:
            str_child_1 = str(self.child_nodes[0])
            str_child_2 = str(self.child_nodes[1])
            if self.node_dist != 0:
                return '(' + str_child_1 + ',' + str_child_2 + '):' + str(round(self.node_dist, 2))
            else:
                return '(' + str_child_1 + ',' + str_child_2 + ')'

def find_min(WM):
    L = len(WM)
    
    cur_i = 0
    cur_j = 1
    cur_min = WM[cur_i][cur_j]

    for i in range(L):
        for j in range(i + 1, L):
            if WM[i][j] < cur_min:
                cur_i = i
                cur_j = j
                cur_min = WM[cur_i][cur_j]
    return cur_i, cur_j, cur_min 

def UPGMA(weight_matrix, names):
    res = [w_node(n) for n in names]

    while len(weight_matrix) > 1:
        i, j, r_min = find_min(weight_matrix)

        full_dist = r_min / 2
            
        names[i] = names[i] + names[j]
        names.pop(j)
        
        L = len(weight_matrix)
                
        # update i row
        for k in range(i + 1, L):
            if k != j:
                
                temp_w = 0
                if k < j:
                    temp_w = weight_matrix[k][j]
                else:
                    temp_w = weight_matrix[j][k]
                                
                weight_matrix[i][k] = (weight_matrix[i][k] * res[i].weight + temp_w * res[j].weight) / (res[i].weight + res[j].weight)
        
        # update i column
        for k in range(0, i):
            weight_matrix[k][i] = (weight_matrix[k][i] * res[i].weight + weight_matrix[k][j] * res[j].weight) / (res[i].weight + res[j].weight)
        
        for k in range(L):
            weight_matrix[k].pop(j)
        
        weight_matrix.pop(j)
        
        res[i] = w_node(names[i], False, res[i], res[j], full_dist)
        res.pop(j)
    
    print(res[0])

def start_test_2(test):
    names_test = [i for i in N[test]]
    weight_matrix_test = [w.copy() for w in W[test]]

    UPGMA(weight_matrix_test, names_test)

Test 1.

In [162]:
test = int(input()) - 1
start_test_2(test)

1
(A:7.0,(B:4.0,(C:2.0,D:2.0):2.0):3.0)


Test 2.

In [163]:
test = int(input()) - 1
start_test_2(test)

2
((((A:2.0,C:2.0):1.0,B:3.0):0.75,(D:2.5,E:2.5):1.25):0.65,F:4.4)
